In [ ]:
!pip install transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import login
import torch

def load_mistral(token: str):
    login(token=token)

    model_id = "mistralai/Mistral-7B-Instruct-v0.1"

    tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=True)
    tokenizer.pad_token = tokenizer.eos_token
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.float16,
        device_map="auto",  # lets Accelerate offload automatically
        use_auth_token=True
    )
    model.eval()
    return model, tokenizer

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import login
import torch

# Prompt builder (friendly, natural)
def build_prompt(topic, role1, emotion1, role2, emotion2):
    prompt = f"""
Write a friendly and natural conversation between two people about the topic "{topic}".

Background:
- {role1} is feeling {emotion1}
- {role2} is feeling {emotion2}

Use any natural-sounding names (not necessarily Alice and Bob).

Only include the dialogue like this:
Person1: ...
Person2: ...
Person1: ...
Person2: ...

give some name to the person(specify what their role is)
Make it casual, like a real chat between two people.
Begin the conversation:
"""
    return prompt.strip()

# Generate conversation
def generate_conversation(model, tokenizer, prompt):
    inputs = tokenizer(prompt, return_tensors="pt", padding=True).to(model.device)
    attention_mask = inputs["attention_mask"]

    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=attention_mask,
        max_new_tokens=700,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Remove prompt from beginning of output (optional)
    trimmed = response.replace(prompt, "").strip()
    return trimmed

# Main
if __name__ == "__main__":
    token = "hugging_face_token"  # Replace with your HF token
    model, tokenizer = load_mistral(token)

    # Get user inputs
    topic = input("Enter the topic of conversation: ").strip()
    role1 = input("Enter Role 1: ").strip()
    emotion1 = input(f"Enter the emotion of {role1}: ").strip()
    role2 = input("Enter Role 2: ").strip()
    emotion2 = input(f"Enter the emotion of {role2}: ").strip()

    # Build and run
    prompt = build_prompt(topic, role1, emotion1, role2, emotion2)
    conversation = generate_conversation(model, tokenizer, prompt)

    print("\n--- Generated Conversation ---\n")
    print(conversation)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:898: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Enter the topic of conversation: first day of college
Enter Role 1: senior
Enter the emotion of senior: nostalgic
Enter Role 2: freshman
Enter the emotion of freshman: nervous

--- Generated Conversation ---

Senior: Hey, (name)! How's it going?

Freshman: (name)! Hi! I'm doing okay. Just a little nervous about my first day of college tomorrow.

Senior: Oh, don't worry! It'll be fine. It's always a little scary on the first day, but it gets better quickly.

Freshman: Yeah, I'm just not sure what to expect. What was your first day like?

Senior: Well, it was a bit chaotic. There were so many people and so much information to take in. But it was also really exciting. I remember feeling like I was finally starting to grow up and take charge of my life.

Freshman: That sounds a little scary, but also kind of fun. I hope it's like that for me too.

Senior: Definitely! And don't worry, you'll make friends. Everyone's a little nervous at first, but it's easy to find people who share your inte